<a href="https://colab.research.google.com/github/LokeRuiKee/timeseries-forecasting-HuggingFace/blob/main/Modified_Read_Values_From_CSV_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read values from closed Excel files 😎

## Imports

In [ ]:
from pathlib import Path  # Standard Python Module
from openpyxl import load_workbook, Workbook  # pip install openpyxl

## 1. Step:
> Get all Excel file paths from the given source directory

In [ ]:
SOURCE_DIR = "/content/data"
excel_files = list(Path(SOURCE_DIR).glob("*.csv"))
excel_files

[PosixPath('/content/data/6_(1227).csv'),
 PosixPath('/content/data/4_(1211).csv'),
 PosixPath('/content/data/2_(1160).csv'),
 PosixPath('/content/data/3_(1218).csv'),
 PosixPath('/content/data/5_(1219).csv')]

## 2. Step:
> Iterate over all Excel files from step 1, access the worksheet and store the values in a **dictionary** <br>
> `values_excel_files = {'2021-01-01' : [1,2,3, ..], '2021-01-02' : [1,2,3, ..], ...}`

In [ ]:
import csv

values_excel_files = {}
for excel_file in excel_files:
    # Extract just the client ID from the filename
    # Assuming the client ID is the number between the parentheses
    client_id = excel_file.stem.split("_")[1][1:-1]  # Extract the number between "_" and ")"
    with open(excel_file, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')  # Specify semicolon as the delimiter
        rng_values = []
        for row in reader:
            # Assuming the data you want is in the third column (index 2)
            # Strip leading and trailing whitespace from the value
            if len(row) >= 3:  # Check if the row has at least three elements
                rng_values.append(row[2].strip())
            else:
                rng_values.append(None)  # Or any other placeholder value if the row is too short
    values_excel_files[client_id] = rng_values

# print(len(values_excel_files))
# print(values_excel_files)

In [ ]:
client_id

'1219'

## 3. Step:
> a) Iterate over all worksheets in the master workbook <br>
> b) For each worksheet, iterate over defined Excel range (dates) <br>
> c) If date matches with the key of dictionary (values_excel_files) then insert values & save workbook

In [ ]:
import csv
from openpyxl import load_workbook
from datetime import datetime

# Load master template
master_template = load_workbook("/content/Masterfile_Template.xlsx")
ws = master_template.active  # Assuming data is on the active sheet

# Initialize dictionary to store client IDs and their corresponding columns
client_columns = {}

# Iterate over CSV files and extract data
for excel_file in excel_files:
    # Extract just the client ID from the filename
    # Assuming the client ID is the number between the parentheses
    client_id = excel_file.stem.split("_")[1][1:-1]  # Extract the number between "_" and ")"
    client_columns[client_id] = len(client_columns) + 3  # Start from column C for client IDs
    with open(excel_file, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')  # Specify semicolon as the delimiter

        # Skip the first three rows
        for _ in range(3):
            next(reader)

        row_index = 3  # Start from row 3 in master template's column B
        for row in reader:
            if len(row) >= 2:  # Check if the row has at least two elements (Date and Time)
                date_str, time_str = row[0].split(";")[0], row[1].split(";")[0]  # Split by semicolon and take the first part as date and time
                time_str = time_str.replace("\t", "").replace(" ", "")  # Remove tabs and spaces from the time string
                if ":" in time_str:
                    hour, minute = time_str.split(":")  # Extract hour and minute if the colon is present
                    time_str = f"{hour.zfill(2)}:{minute.zfill(2)}"  # Construct time string with leading zeros if necessary
                else:
                    time_str = time_str.zfill(4)  # If colon is missing, assume format HHMM and pad with leading zeros
                    hour, minute = time_str[:2], time_str[2:]  # Extract hour and minute
                    time_str = f"{hour}:{minute}"  # Construct time string with colon
                # Combine date and time strings into a single datetime object
                date_time = datetime.strptime(date_str + " " + time_str, "%d.%m.%Y %H:%M")
                # Fill date into column B for each client ID
                ws.cell(row=row_index, column=2, value=date_time)  # Timestamp in column B
                # Assuming the data you want is in the third column (index 2)
                # Strip leading and trailing whitespace from the value and replace commas with periods
                if len(row) >= 3:  # Check if the row has at least three elements
                    value = row[2].strip().replace(",", ".")
                else:
                    value = None  # Or any other placeholder value if the row is too short
                ws.cell(row=row_index, column=client_columns[client_id], value=value)  # Watt data in respective client column
                row_index += 1  # Move to the next row in the master template

# Fill client IDs in row 2 starting from column C
for client_id, column_index in client_columns.items():
    ws.cell(row=2, column=column_index, value=client_id)

# Optionally, save the master template with the filled data
master_template.save("/content/Masterfile_filled.xlsx")